In [7]:
setwd("/home/pa/Documents/github/doc_suomi/code")
source("utils.R")
source("data_cook.R")
cat(howto_data)

`summarise()` has grouped output by 'album_id', 'name'. You can override using the `.groups` argument.



How to use datasets (Personal library)
 
call    base()             for real values
call    z_scored()         for normalized
call    min_maxed()        for normalized2
call    upsampled_album()  for binded upsamplped albums
call    low_minmax()       for min-maxed low level
call    low_z()            for normalized low level
call    low_raw()          for raw  low level

# Dissimilarities 1

Dissimilarities with nearest neighbors upsampling. Each album was upsampled to 16 tracks, and then dissimilarities were calculated.

In [ ]:
# Reading upsampled albums
file_list <- list.files(path="/home/pa/Documents/github/doc_suomi/data/dissimilarities/upsampled_nearest")
matrices = c()

# Reading files
for(i in 1:length(file_list)){
    file = paste("/home/pa/Documents/github/doc_suomi/data/dissimilarities/upsampled_nearest/", file_list[i], sep = "")
    matrices[[i]] = fread(file, header = TRUE)
}

matrices[[3831]] <- NULL

for(i in 1:length(matrices)){
    colnames(matrices[[i]]) <- c("track_number", "album_id", "valence", "energy", "loudness", "tempo")
    matrices[[i]]$album_id <- as.character(matrices[[i]]$album_id)
    matrices[[i]]$track_number <- c(seq(1, 16, 1))
}

matrices = dissim_gen(matrices)

dt <- matrix_parser(matrices,  function(x){mean(x, na.rm = TRUE)})
std <- matrix_parser(matrices, function(x){sd(x, na.rm = TRUE)/sqrt(length(x))})

redblack = colorRampPalette(c("red", "black"))(100)
levelplot(dt, col.regions = redblack)

# Dissimilarities 2

Each album was upsampled to 100 tracks with nearest neighbors. Then dissimilarities were calculated.

In [ ]:
# Reading upsampled albums
file_list <- list.files(path="/home/pasoneto/Documents/github/doc_suomi/data/dissimilarities/upsampled_100")
matrices = c()

# Reading files
for(i in 1:length(file_list)){
    file = paste("/home/pasoneto/Documents/github/doc_suomi/data/dissimilarities/upsampled_100/", file_list[i], sep = "")
    matrices[[i]] = fread(file, header = TRUE)
}

matrices[[3831]] <- NULL

for(i in 1:length(matrices)){
    colnames(matrices[[i]]) <- c("track_number", "album_id", "valence", "energy", "loudness", "tempo")
    matrices[[i]]$album_id <- as.character(matrices[[i]]$album_id)
    matrices[[i]]$track_number <- c(seq(1, 100, 1))
}

matrices = dissim_gen(matrices)

dt <- matrix_parser(matrices,  function(x){mean(x, na.rm = TRUE)})
std <- matrix_parser(matrices, function(x){sd(x, na.rm = TRUE)/sqrt(length(x))})

redblack = colorRampPalette(c("red", "black"))(100)
levelplot(dt, col.regions = redblack)

In [8]:
library("tidyverse")

#Import list of dissimilarities
oi = list_upsampled() #upsampled using bilinear interpolation

#Renaming columns and resetting track_number
for(i in 1:length(oi)){
    oi[[i]]$`0` <- seq(1, 16, 1)
    colnames(oi[[i]]) <- c("track_number", "album_id", "valence", "energy", "loudness", "tempo")
}

#Generating dissimilarities
oi = dissim_gen(oi)

# #Getting dissimilarity from each album
# #each column represents the dissimilarity of track n with every other track
# dissims = c()
# for(k in 1:16){
#     track = c()
#     for(i in 1:length(oi)){ track <- c(track, oi[[i]][k, ])}
#     dissims[[k]] <- unname(track)
# }

# #Generating data frame for visuals
# names = c(); value = c()
# for(i in 1:length(dissims)){
#     names = c(names, rep(paste("track", i, sep = ""), 71120))
#     value = c(value, dissims[[i]])    
# }

# visual_dissim = tibble(dissims = value, track_ref = names)

# # Plotting
# dp = visual_dissim %>%
#     na.omit() %>% 
#     mutate(track_ref = fct_reorder(.f = track_ref, .x = dissims, .fun = median)) %>% 
#     ggplot(aes(x = dissims, y = track_ref, group = track_ref)) +
#            ggridges::stat_density_ridges(aes(fill = track_ref),
#                                          rel_min_height = 0.03, 
#                                          scale = 5, 
#                                          quantile_lines = TRUE, 
#                                          quantiles = 2)+
#     theme(legend.position = "none") + xlim(c(0, 1.6))

# bp = visual_dissim %>%
#     na.omit() %>% 
#     mutate(track_ref = fct_reorder(.f = track_ref, .x = dissims, .fun = median)) %>% 
#     ggplot(aes(y = dissims, x = track_ref, group = track_ref)) +
#            geom_boxplot(aes(fill = track_ref)) +
#     theme(axis.text.x = element_text(angle = 60))

# pp = visual_dissim %>%
#     na.omit() %>% 
#     group_by(track_ref) %>%
#     summarise(dissim = mean(dissims), stder = sd(dissims)/sqrt(length(dissims))) %>%
#     mutate(track_ref = fct_reorder(.f = track_ref, .x = dissim, .fun = mean)) %>% 
#     ggplot(aes(x = track_ref, y = dissim, fill = track_ref))+
#         geom_point(size = 0.5)+
#         geom_errorbar(aes(ymin = dissim-stder, ymax = dissim+stder)) +
#     theme(axis.text.x = element_text(angle = 60))

ERROR: Error in fread(file, header = TRUE): File '/home/pasoneto/Documents/github/doc_suomi/data/dissimilarities/upsampled_bilinear/matriz1.csv' does not exist or is non-readable. getwd()=='/home/pa/Documents/github/doc_suomi/data/treated_data'


# Dissimilarities 3

Here I'm looking at overall dissimilarities for each track. Essentially what I'm asking is: "how dissimilar is track $n$ from all other tracks?"

## Mean and standard error of dissimilarities by track

In [ ]:
pp

## Boxplot of dissimilarities by track

In [ ]:
bp

# Dissimilarities by album length

Here I'm asking if the pattern of highest dissimilarities hold for albums regardless of their length. For instance, albums with 6 and albums with 16 tracks show the highest dissimilarities on their edges? 

Graphs are displayed in order from album lengths of 6 to 16

In [ ]:
dt = base()
dt %<>% split(dt$album_length)
dt = dissim_by_length(dt)

In [ ]:
for(i in 1:length(dt)){
    dt[[i]] <- matrix_parser(dt[[i]], function(x){mean(x, na.rm = TRUE)})
    print(levelplot(dt[[i]], col.regions = colorRampPalette(c("red", "black"))(100)))
}

#  Down sampling to the smallest album length

Down sampled every album to 6 tracks

In [ ]:
# Reading upsampled albums
file_list <- list.files(path="/home/pasoneto/Documents/github/doc_suomi/data/dissimilarities/down_sampled")
matrices = c()

# Reading files
for(i in 1:length(file_list)){
    file = paste("/home/pasoneto/Documents/github/doc_suomi/data/dissimilarities/down_sampled/", file_list[i], sep = "")
    matrices[[i]] = fread(file, header = TRUE)
}

matrices[[3831]] <- NULL

for(i in 1:length(matrices)){
    colnames(matrices[[i]]) <- c("track_number", "album_id", "valence", "energy", "loudness", "tempo")
    matrices[[i]]$album_id <- as.character(matrices[[i]]$album_id)
    matrices[[i]]$track_number <- c(seq(1, 6, 1))
}

matrices = dissim_gen(matrices)

dt <- matrix_parser(matrices,  function(x){mean(x, na.rm = TRUE)})
std <- matrix_parser(matrices, function(x){sd(x, na.rm = TRUE)/sqrt(length(x))})

redblack = colorRampPalette(c("red", "black"))(100)
levelplot(dt, col.regions = redblack)